Import lib

In [ ]:
import numpy as np

Code donné pour la fonction d'entrainement du l'algo



In [12]:
def h(x):
  if x > 0:
    return 1
  else:
    return 0

def neuron_train(X, Y, w, µ):
  it      = 0
  is_over = False
  while is_over is False:
    is_over = True

    for x, y_ex in zip(X, Y):
      a = np.inner(w, x) # vector product
      y = h(a)
      e = y_ex - y

      w = w + µ * e * x

      if e != 0:
        is_over = False

    it += 1

  return w, it

w = np.array([0.3, 0.1, -0.2])
X = np.array([ # we add 1 to represent the bias
  [1, 0, 0],
  [1, 1, 0],
  [1, 0, 1],
  [1, 1, 1]
])
y = np.array([0, 1, 1, 1])
µ = 0.5

w, it = neuron_train(X, y, w, µ)
print(w, it)

[-0.2  0.6  0.3] 4


Fonction qui sert à générer des données

In [13]:
from torch.utils.data import TensorDataset
import math

def generate_dataset(n_samples):
  # nous générons des vecteurs de taille 2 dont les valeurs sont entre -1 et 1
  x = torch.rand(n_samples, 2) * 2 - 1

  # nous calculons la distance entre le point et l'origine
  x_norms = (x ** 2).sum(axis = 1).sqrt()

  # le rayon du disque
  radius = math.sqrt(2 / math.pi)

  # compare la norme au rayon du disque, créé un masque booléen,
  # puis trasnforme en entier 8 octects.
  y = (x_norms < radius).long()

  return TensorDataset(x, y)

Création d'un modèle perceptron multicouche. Il y a deux couches de 64 neurones

In [21]:
import torch
import torch.nn as nn

class MLP(nn.Module):
  def __init__(self):
    super(MLP, self).__init__()
    self.hidden_layer_1    = nn.Linear(2, 64)       #2 connexions en entrée 64 en sortie
    self.hidden_layer_2    = nn.Linear(64, 64)      #64 connexions en entrée 64 en sortie
    self.output_layer      = nn.Linear(64, 2)       #64 connexions en entrée 2 en sortie
    self.hidden_transfer_1 = nn.ReLU()
    self.hidden_transfer_2 = nn.ReLU()
    self.output_transfer   = nn.LogSoftmax(dim = 1)

  def forward(self, x):
    x = self.hidden_layer_1(x)
    x = self.hidden_transfer_1(x)
    x = self.hidden_layer_2(x)
    x = self.hidden_transfer_2(x)
    x = self.output_layer(x)
    x = self.output_transfer(x)

    return x

Test de précion du résultat de chaque neurne

In [22]:
from torch.utils.data import DataLoader

def train(
  model, loss_func, optimizer, train_dataset,
  test_dataset, epochs, batch_size
):
  dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)

  for epoch in range(epochs):
    for X, y in dataloader:
      y_pred = model(X)
      loss   = loss_func(y_pred, y)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

    if epoch % 10 == 0:
      train_accuracy = evaluate(model, train_dataset, batch_size)
      test_accuracy  = evaluate(model, test_dataset, batch_size)
      print(
        f'{epoch:3} -> {100 * train_accuracy:5.3f}% train accuracy',
        f'{epoch:3} -> {100 * test_accuracy:5.3f}% test accuracy'
      )

In [25]:
def evaluate(model, dataset, batch_size):
  dataloader = DataLoader(dataset, batch_size = batch_size, shuffle = False)

  correct_pred = 0
  total_pred   = 0
  with torch.no_grad():
    for X, y in dataloader:
      y_pred        = model(X)
      y_pred_class  = y_pred.argmax(dim = 1)
      correct_pred += (y_pred_class == y).sum().item()
      total_pred   += len(y)

  return correct_pred / total_pred

Initialisation du modèle d'entrainement. Exécution du modèle



In [26]:
import torch.optim as optim

n_samples     = 3000
epochs        = 300
batch_size    = 32
learning_rate = 1e-3

train_dataset = generate_dataset(n_samples)
test_dataset  = generate_dataset(n_samples)
model         = MLP()
loss_func     = nn.NLLLoss()
optimizer     = optim.SGD(
  params = model.parameters(),
  lr     = learning_rate
)
train(
  model, loss_func, optimizer, train_dataset,
  test_dataset, epochs, batch_size
)

  0 -> 49.367% train accuracy   0 -> 50.467% test accuracy
 10 -> 61.500% train accuracy  10 -> 61.567% test accuracy
 20 -> 67.833% train accuracy  20 -> 67.167% test accuracy
 30 -> 71.633% train accuracy  30 -> 70.767% test accuracy
 40 -> 76.967% train accuracy  40 -> 76.267% test accuracy
 50 -> 81.800% train accuracy  50 -> 80.200% test accuracy
 60 -> 84.167% train accuracy  60 -> 83.167% test accuracy
 70 -> 86.733% train accuracy  70 -> 86.500% test accuracy
 80 -> 89.167% train accuracy  80 -> 89.200% test accuracy
 90 -> 90.800% train accuracy  90 -> 91.033% test accuracy
100 -> 92.533% train accuracy 100 -> 92.433% test accuracy
110 -> 93.733% train accuracy 110 -> 93.100% test accuracy
120 -> 94.333% train accuracy 120 -> 93.933% test accuracy
130 -> 94.700% train accuracy 130 -> 94.433% test accuracy
140 -> 95.133% train accuracy 140 -> 94.900% test accuracy
150 -> 95.367% train accuracy 150 -> 95.167% test accuracy
160 -> 95.800% train accuracy 160 -> 95.900% test accura